# Get data from cosmos db containers

In [2]:
# Champion Container
readConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
"Database" : "league_all_data",
"Collection" : "champions", 
}
champ = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**readConfig).load()

In [3]:
# match container
matchConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
"Database" : "league_all_data",
"Collection" : "matches_final", 
}

match = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**matchConfig).load()

In [4]:
#match filter
match.createOrReplaceTempView("table1")
srmatch=spark.sql("SELECT * from table1 where stats.gameMode='CLASSIC' and stats.gameDuration>420") #filtered match table

In [5]:
#player container
playConfig = {
"Endpoint" : "https://lolpredict.documents.azure.com:443/",
"Masterkey" : "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
"Database" : "league_all_data",
"Collection" : "players_final", 
}

play = spark.read.format("com.microsoft.azure.cosmosdb.spark").options(**playConfig).load()

In [6]:
#Total counts of raw data queried
play_count=play.count()
match_count=match.count()
print("total players queried is:" + str(play_count))
print("total matches queried is:" + str(match_count))

total players queried is:9784
total matches queried is:25784

In [7]:
#player filtering/transforming
import pyspark.sql.functions as f
play.createOrReplaceTempView("table2")
play2=spark.sql("SELECT mastery, id from table2")
# play2 = play2.limit(1650)

# Champion Table

In [9]:
#Getting the champion statistics we intend to use for the final model
champ.createOrReplaceTempView("table3")
champ2=spark.sql("SELECT id, name, stats.armorperlevel, stats.attackdamageperlevel, stats.attackspeed, stats.attackrange, stats.spellblockperlevel, stats.hpperlevel FROM table3")
champ2.count()

In [10]:
mast_champ_join = champ2

# Get label vector

In [12]:
#Engineering the target vector
import pyspark
label2 = srmatch.select(pyspark.sql.functions.col('id'),
                       pyspark.sql.functions.col('stats.teams.teamId').alias('teamId'),
                       pyspark.sql.functions.col('stats.teams.win').alias('win'))

In [13]:
#UDF to change win feature into a binary 0/1 feature
from pyspark.sql.types import IntegerType, ArrayType
from pyspark.sql.functions import udf
def create_label(row):
    if row[0]=='Win':
        return 1
    return 0
label_udf = udf(create_label, IntegerType())
label2 = label2.withColumn('label', label_udf(label2.win))
label2.show(10)

In [14]:
label_col = label2.drop('teamId','win')
label_col.show(2)

## Create stats table

### Exploded match table

In [17]:
#Exploding the player match histories to get individual match ids
from pyspark.sql.functions import split, explode

matches_view =spark.sql("SELECT id, matches FROM table2")
match_sum_tab=matches_view.withColumn('matchId', explode('matches'))
match_sum_tab.show(3)

### Get stats for each match

In [19]:
import numpy as np

In [20]:
matches_ids = srmatch.select("id")
matchIds_list = np.array(matches_ids.collect())
matchIds_list = [str(i[0]) for i in matchIds_list] #ids of all matches 

In [21]:
matchDur =  srmatch.select("stats.gameDuration")
matchDur_list = np.array(matchDur.collect())
matchDur_list = [float(i[0])/60 for i in matchDur_list] #game length for all matches in minutes 

In [22]:
all_matches_stats = srmatch.select(
    'stats.participantIdentities.player.summonerId',
                     'stats.participants.championId',
                     'stats.participants.timeline.lane',
                     'stats.participants.timeline.role',
                    'stats.participants.teamId',
                    'stats.participants.stats.goldEarned',
                    'stats.participants.stats.kills',
                    'stats.participants.stats.assists',
                    'stats.participants.stats.deaths',
                    'stats.participants.stats.visionScore',
                    'stats.participants.stats.totalDamageDealt',
                    'stats.participants.stats.totalDamageTaken',
                    'stats.participants.stats.totalMinionsKilled',
                    'stats.participants.stats.totalTimeCrowdControlDealt')

#stats of all matches
stats_np = np.array(all_matches_stats.collect())

In [23]:
all_match_dict = []
for match,stats,game_length in zip(matchIds_list,stats_np,matchDur_list):
    #combine match stats with stats id to create a new dataframe
    match_dict = {}
    match_dict['match_id']= match
    match_dict['match_length'] = game_length
    summonerIds = stats[:][0]
    champions = stats[:][1]
    lanes = stats[:][2]
    roles = stats[:][3]
    teamId = stats[:][4]
    avg_stats = stats[:][5:]
    avg_stats_dice = [list(i) for i in zip(*avg_stats)]
    match_dict['agg_stats'] = [list(map(float,i)) for i in avg_stats_dice]
    match_dict['lanes'] = [str(i) for i in lanes]
    match_dict['roles'] = [str(i) for i in roles]
    match_dict['teamId'] = [str(i) for i in teamId]
    match_dict['summonerIds'] = [str(i) for i in summonerIds]
    match_dict['champions'] = [str(i) for i in champions]
    all_match_dict.append(match_dict)


In [24]:
match_sum_stats_agg_df = spark.createDataFrame(all_match_dict)
match_sum_stats_agg_df.show(2)

In [25]:
from pyspark.sql.types import *
from pyspark.sql.functions import col, udf, explode

zip_ = udf(
  lambda a,b,c,d,e,f: list(zip(a,b,c,d,e,f)),
  ArrayType(StructType([
      # Adjust types to reflect data types
      StructField("stats", ArrayType(DoubleType())),
      StructField("champion", StringType()),
      StructField("summonerIds", StringType()),
    StructField("lane", StringType()),
    StructField("role", StringType()),
    StructField("team", StringType()),           
  ]))
)

In [26]:
#explode table to create a more fine grain accessing method 
match_sum_stats_s = match_sum_stats_agg_df.withColumn("temp",zip_("agg_stats","champions","summonerIds","lanes","roles","teamId"))
match_sum_stats_s = match_sum_stats_s.withColumn("temp", explode("temp"))

In [27]:
match_sum_stats = match_sum_stats_s.select("match_id",col("temp.stats").alias("player_stats_inv"), 
                         col("temp.champion").alias("champion_inv"),col("temp.summonerIds").alias("summonerId_inv"),
                         col("temp.lane").alias("lanes_inv"),
                         col("temp.role").alias("roles_inv"),
                         col("temp.team").alias("team"),"match_length")
match_sum_stats.show(10)

### clean role table (merge role, lane and team)

In [29]:
def processed_role(lane,role,team):
    final_role =""
    if lane != "BOTTOM":
        final_role = "{}_{}".format(team,lane)
    else:
        if "SUPPORT" in role:
            final_role = "{}_SUPPORT".format(team)
        elif "CARRY" in role:
            final_role = "{}_CARRY".format(team)
        else:
            final_role = "CANT"
    if "NONE" in final_role:
        final_role = "CANT"
    return final_role
udf_clean_role = zip_ = udf(
  lambda lane,role,team: processed_role(lane,role,team), StringType()
)
match_sum_stats = match_sum_stats.withColumn('final_role', udf_clean_role("lanes_inv", "roles_inv","team"))
match_sum_stats = match_sum_stats.drop("lanes_inv","roles_inv","team")
match_sum_stats = match_sum_stats.select("*").filter(match_sum_stats.final_role != "CANT") #remove players without clear role
match_sum_stats.show(10)

### normalize stats (divide by game length in minutes)

In [31]:
def normalize_stats(stats,game_length):
    return [ float(stat/game_length) for stat in stats]
    
normalize_udf = udf(normalize_stats,ArrayType(FloatType()))
match_sum_stats_2 = match_sum_stats.withColumn('normalize_stats', normalize_udf(match_sum_stats.player_stats_inv, match_sum_stats.match_length))

In [32]:
match_sum_stats_final = match_sum_stats_2.drop("player_stats_inv") #drop non-normalize stats => we only consider stats/minute

In [33]:
match_sum_stats_final.show(2)

## Explode players stats

In [35]:
#explode the normalize stats column to multiple columns
match_sum_stats_final = match_sum_stats_final.select("match_id","champion_inv","summonerId_inv","final_role",
                                             match_sum_stats_final.normalize_stats[0].alias("gold_earned"),
                                             match_sum_stats_final.normalize_stats[1].alias("kills"),
                                             match_sum_stats_final.normalize_stats[2].alias("assists"),
                                             match_sum_stats_final.normalize_stats[3].alias("deaths"),
                                             match_sum_stats_final.normalize_stats[4].alias("visionScore"),
                                             match_sum_stats_final.normalize_stats[5].alias("totalDamageDealt"),
                                             match_sum_stats_final.normalize_stats[6].alias("totalDamageTaken"),
                                             match_sum_stats_final.normalize_stats[7].alias("totalMinionsKilled"),
                                             match_sum_stats_final.normalize_stats[8].alias("totalTimeCrowdControlDealt"))

In [36]:
match_sum_stats_final.show(1)

## Create average stats table

In [38]:
avg_stats = match_sum_stats_final.select("summonerId_inv","gold_earned","kills",                      
             "assists","deaths","visionScore","totalDamageDealt","totalDamageTaken","totalMinionsKilled",
            "totalTimeCrowdControlDealt")


In [39]:
avg_stats.show(2)

In [40]:
import pyspark.sql.functions as F
summoner_avg_stats = avg_stats.groupBy('summonerId_inv').agg(F.avg('gold_earned'),F.avg('kills'),F.avg("deaths"),
                                     F.avg('assists'),F.avg('visionScore'),F.avg('totalDamageDealt'),
                                     F.avg('totalDamageTaken'),F.avg('totalMinionsKilled'),
                                     F.avg('totalTimeCrowdControlDealt'))

In [41]:
summoner_avg_stats.show(1)

### summonerID_avgstats_matchId table

In [43]:
mast_champ_join.show(2)

In [44]:
match_sum_stats_join = match_sum_stats_final.select("summonerId_inv","match_id","champion_inv","final_role")
match_sum_stats_join.show(2)

In [45]:
match_sum_stats_join = match_sum_stats_join.join(mast_champ_join,[mast_champ_join.id==match_sum_stats_join.champion_inv])

In [46]:
match_sum_stats_join = match_sum_stats_join.drop("id")
match_sum_stats_join.show(2)

### join the final features table for each individual for each match

In [48]:
player_stats_final = match_sum_stats_join.join(summoner_avg_stats,[
        match_sum_stats_join.summonerId_inv == summoner_avg_stats.summonerId_inv,
    ]).drop(match_sum_stats_join.summonerId_inv)

In [49]:
# player_stats_final = player_stats_final.drop("summonerId_inv")
player_stats_final.printSchema()

In [50]:
match_with_data = player_stats_final.groupby('match_id').count().select("match_id",F.col('count').alias('number_of_players'))

In [51]:
#join with current all players stats player

In [52]:
players_matchs_stats = player_stats_final.join(match_with_data,[match_with_data.match_id == player_stats_final.match_id]).drop(player_stats_final.match_id)

In [53]:
players_matchs_stats = players_matchs_stats.drop("match_with_data.match_id")

In [54]:
players_matchs_stats.printSchema()

In [55]:
players_matchs_stats.show(1)

### remove matches with less than 10 players data

In [57]:
#We only want matches where every player was queried in our player data so that we cna get all of their average statistics
players_matchs_stats = players_matchs_stats.filter(players_matchs_stats.number_of_players == 10)
players_matchs_stats.count()

In [58]:
players_matchs_stats.show(2)

## join with label table

In [60]:
label_col.show(1)

In [61]:
# label = 100_win or not
players_stats_sep_table = players_matchs_stats.join(label_col,[label_col.id == players_matchs_stats.match_id])

In [62]:
players_stats_sep_table.show(1)

In [63]:
all_stats_matchs_pd = players_stats_sep_table.toPandas()

# Create feature matrix

In [65]:
import pandas as pd
all_stats_matchs_pd = all_stats_matchs_pd.drop(['id','number_of_players'],axis=1)
matchs_group = all_stats_matchs_pd.groupby("match_id").groups

In [66]:
print("there are {} legit matches for final matrix".format(len(matchs_group)))

In [67]:
one_match = all_stats_matchs_pd.iloc[matchs_group['3207612018']]
feat_list = list(one_match.columns) #list of features
feat_list 

In [68]:
features_matrix_list = []
for group_key in matchs_group.keys():
    one_match = all_stats_matchs_pd.iloc[matchs_group[group_key]]
    feat_count = 2
    match_dict = {} #create a dict for 1 match 
    match_dict['label'] = int(one_match.iloc[0]['label'])
    match_dict['match_id'] = str(one_match.iloc[0]['match_id'])
    for user in one_match.iterrows():
        player = user[1]
        role = player['final_role']
        processed = ['final_role','label','match_id']
        string_feat = ['summonerId_inv','champion_inv','name']
        int_feat = ['label']
        for feat in feat_list:
            if feat not in processed:
                if feat in string_feat:
                    val = str(player[feat])
                else:
                    val = float(player[feat])
                match_dict["{}_{}".format(role,feat)] = val
                feat_count += 1
    if len(match_dict) == 182:
        features_matrix_list.append(match_dict)

In [69]:
print("after cleaning data, we have {} matches left".format(len(features_matrix_list)))

# Final feature matrix

In [71]:
myJson = sc.parallelize(features_matrix_list)
features_matrix = sqlContext.read.json(myJson) #change list of dict to pyspark dataframe

In [72]:
features_matrix.count()

## save feature matrix to CosmosDB

In [74]:
writeConfig = {
    "Endpoint": "https://lolpredict.documents.azure.com:443/",
    "Masterkey": "lAb4WLi89cpXb7jGeMlLZtXUqvlGJip9NtKxDZMWHKUf0wasuAFknSgTxX3M7RAlK2uyFZjau7AJethdaBRDWw==",
    "Database": "league_extraction",
    "Collection": "FINAL_FEATURES_MATRIX_9112",
    "Upsert": "true"
}

# Write to Cosmos DB from the flights DataFrame
features_matrix.write.format("com.microsoft.azure.cosmosdb.spark").options(
    **writeConfig).save()

In [75]:
9112 